In [ ]:
from tqdm import tqdm

import numpy as np
import pandas as pd

import yfinance as yf

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

risk_free_rate = 0.03
trial_n = 100

tickers = ['NKE', 'MSFT', 'DIS', 'COST']

#start = '2012-1-1'
#end = '2022-3-1'
start = '2012-1-1'
end = '2022-1-1'


df_close = yf.download(tickers, start, end, interval='1mo')['Close']

df_close.to_csv('./data/df_close.csv')

In [ ]:
df_close.shape

In [ ]:
df_close.head()

In [ ]:
df_close.tail()

In [ ]:
df_close.isnull().sum()

In [ ]:
if df_close.isnull().values.any():
    df_close.fillna(method='ffill', inplace = True) #front fills
    df_close.fillna(method='bfill', inplace = True) #back fills

In [ ]:
def plot(df, xlabel, ylabel, title):
    #matplotlib
    ax = df_close.plot(title = title, figsize=(16, 8), ax = None)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.axhline(y=0, color = 'black')
    ax.legend(loc='upper left')
    plt.show()

    #plotly
    fig = px.line(df_close, title = title)
    fig.update_layout(xaxis_title=xlabel, yaxis_title=ylabel,showlegend=True, template='seaborn')
    fig.show()

In [ ]:
plot(df_close, 'Date', 'Stock Price', 'Close Price')

In [ ]:
df_pct_change = df_close.pct_change().iloc[1:] #get rid of last row
df_pct_change.head()

In [ ]:
plot(df_pct_change, 'Date', 'Percent', 'Percent Change (monthly return)')

In [ ]:
mean_return = df_pct_change.mean() * 12
mean_return

In [ ]:
var_risk = df_pct_change.var() * 12
var_risk

In [ ]:
cov_mat = df_pct_change.cov()*12
cov_mat

In [ ]:
corr_mat = df_pct_change.corr()
corr_mat

In [ ]:
fig = px.imshow(corr_mat, title='Correlation Heatmap')
fig.show()

In [ ]:
weight = 1/len(tickers)
eq_weights = [weight for i in tickers]
eq_weights

In [ ]:
eq_return = eq_weights @ (mean_return)
eq_return

In [ ]:
eq_risk = ((eq_weights @ cov_mat) @ eq_weights)**(1/2)
eq_risk

In [ ]:
eq_sharpe = (eq_return - risk_free_rate)/eq_risk
eq_sharpe

In [ ]:
efficient_frontier_data = []
for trial in tqdm(range(trial_n)):
    t_weight = np.random.random(len(tickers, ))
    t_weights = t_weight/sum(t_weight)
    t_return = t_weights @ (mean_return)
    t_risk = ((t_weights @ cov_mat) @ t_weights)**(1/2)
    t_sharpe = (t_return - risk_free_rate)/t_risk
    efficient_frontier_data.append([t_weights, t_risk, t_return, t_sharpe])

efficient_frontier_data = pd.DataFrame(efficient_frontier_data, columns=['Weights', 'Risk', 'Return', 'Sharpe'])
efficient_frontier_data.head()

In [ ]:
i_max_sharpe = efficient_frontier_data['Sharpe'].idxmax()
max_sharpe = efficient_frontier_data.iloc[i_max_sharpe, :]
max_sharpe

In [ ]:
i_min_risk = efficient_frontier_data['Risk'].idxmin()
min_risk = efficient_frontier_data.iloc[i_min_risk, :]
min_risk

In [ ]:
i_max_return = efficient_frontier_data['Return'].idxmax()
max_return = efficient_frontier_data.iloc[i_max_return, :]
max_return

In [ ]:
print(eq_risk, eq_return)
print(max_sharpe['Risk'], max_sharpe['Return'])


In [ ]:
from plotly import graph_objects as go


def addMarker(fig, x, y, color):
    fig.add_trace(
        go.Scatter(
            x=[x],
            y=[y],
            mode="markers",
            marker_symbol='star',
            marker_size=10,
            marker_color=color
        )
    )


In [ ]:
fig = px.scatter(efficient_frontier_data, x='Risk', y='Return', title='Efficient Frontier', color='Risk')
addMarker(fig, max_sharpe['Risk'], max_sharpe['Return'], 'Green')
addMarker(fig, min_risk['Risk'], min_risk['Return'], 'Cyan')
addMarker(fig, max_return['Risk'], max_return['Return'], 'Red')
fig.show()

## Process data for recent past year for same stocks and the spy

In [ ]:
tickers = ['NKE', 'MSFT', 'DIS', 'COST']

start = '2022-1-1'
end = '2023-1-1'

df_recent_close = yf.download(tickers, start, end, interval='1d')['Close']
df_recent_close.to_csv('./data/df_recent_close.csv')

df_spy_close = yf.download('SPY', start, end, interval='1d')['Close']
df_spy_close.to_csv('./data/df_spy_close.csv')

## Percent Change for recent past year

In [ ]:
recent_pct_change = df_recent_close.pct_change().iloc[1:] #get rid of last row
recent_mean_return = recent_pct_change.mean() * 365
recent_risk = recent_pct_change.var() * 365 

In [ ]:
plot(recent_pct_change, 'Date', 'Percent', 'Percent Change (daily return)')

In [ ]:
spy_pct_change = df_spy_close.pct_change().iloc[1:] #get rid of last row
spy_mean_return = spy_pct_change.mean() * 365
spy_risk = spy_pct_change.var() * 365

In [ ]:
plot(spy_pct_change, 'Date', 'Percent', 'Percent Change (daily return)')

## Cumulative Product Past Recent Year

In [ ]:
recent_cumulative_product = recent_pct_change.add(1).cumprod().sub(1)

In [ ]:
plot(recent_cumulative_product, 'date', 'cumulative product', 'cumulative product' )

In [ ]:
spy_cumulative_product = spy_pct_change.add(1).cumprod().sub(1)

In [ ]:
plot(spy_cumulative_product, 'date', 'cumulative product', 'cumulative product')